<a href="https://colab.research.google.com/github/idowujames/-Kenya-Clinical-Reasoning-Challenge/blob/main/Kenya_Clinical_Reasoning_Challenge_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We begin to train our model, starting with by fine-tunning using BioGPT

In [1]:
# Install sacremoses for BioGPT
!pip install sacremoses --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 7.8 MB/s eta 0:00:00


In [2]:
import pandas as pd

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    AutoModelForCausalLM
    )

### Loading the dataset

In [3]:
df = pd.read_csv('cleaned_dataset.csv')

print(df.shape)
dataset = Dataset.from_pandas(df)

(386, 14)


### Dataset splitting

In [4]:
# Using 90% for training and 10% for validation
train, val = dataset.train_test_split(test_size=0.1, seed=42).values()

print(train.shape)
print(val.shape)

(347, 14)
(39, 14)


### Tokenization

In [5]:
model_name = 'microsoft/biogpt'

tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

In [6]:
# Creating a tokenizer function
def tokenize_function(examples):
    # For each example, we combine the prompt and clinician response.
    # We use the tokenizer's end-of-sentence token as a separator and at the very end.
    # This teaches the model the full structure of a prompt-response pair.
    text_with_format = [prompt + tokenizer.eos_token + response + tokenizer.eos_token
                        for prompt, response in zip(examples['Prompt'], examples['Clinician'])]


    return tokenizer(text_with_format, truncation=True, max_length=256)


# Appling the function to the entire dataset.
tokenized_train_dataset = train.map(
    tokenize_function,
    batched=True,
    remove_columns=train.column_names
)

tokenized_test_dataset = val.map(
    tokenize_function,
    batched=True,
    remove_columns=val.column_names
)

print("Tokenization Complete")
print(tokenized_train_dataset[0])

Map:   0%|          | 0/347 [00:00<?, ? examples/s]

Map:   0%|          | 0/39 [00:00<?, ? examples/s]


--- Tokenization Complete ---
Example of the first tokenized training sample:
{'input_ids': [2, 390, 3175, 14, 4064, 15, 249, 113, 5, 854, 10, 593, 1790, 2005, 10, 14, 150, 6362, 10, 803, 3880, 1167, 4741, 14, 13471, 10, 18559, 14068, 390, 47, 566, 14, 7946, 149, 21, 1061, 113, 421, 8, 6, 3660, 5209, 14, 757, 5, 3109, 41, 475, 606, 13, 7051, 8712, 16, 6, 1337, 86, 876, 5210, 57, 102, 1354, 353, 6, 7552, 21, 2095, 6, 3909, 1622, 31, 175, 597, 826, 390, 6327, 35, 11219, 826, 390, 2971, 14, 4514, 8081, 24135, 30, 826, 390, 25112, 27266, 10343, 30, 826, 390, 14285, 2, 3885, 14, 348, 206, 421, 7946, 34, 803, 3880, 1167, 4741, 14, 150, 1594, 17, 7990, 10, 23, 3109, 3660, 149, 1238, 6, 1284, 57, 41, 9899, 8712, 16, 6, 1694, 86, 876, 6, 1284, 57, 102, 1354, 353, 61, 2321, 15, 14, 2095, 7552, 11058, 2306, 31, 175, 36, 597, 826, 390, 6327, 35, 125, 18872, 579, 30, 3339, 27266, 10343, 13, 14053, 2281, 6, 8712, 2971, 32, 24135, 13, 1590, 6, 8712, 387, 598, 1003, 12583, 6, 1284, 279, 940, 13, 1022

### Training our model

In [7]:
# Fine-Tuning the BioGPT Model with our data

model = AutoModelForCausalLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=50,
    weight_decay=0.01,               # Regularization to prevent overfitting
    logging_steps=10,

    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,

    report_to="none",
)

# The data collator is a helper function that takes the tokenized samples and
# batches them together, handling padding dynamically.
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initializing the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,         # The dynamic padding helper
)

# Starting the Fine-Tuning
print("--- Starting Fine-Tuning ---")
trainer.train()

# 6. Save the fine-tuned model
final_model_path = "./final_biogpt_model"
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
print(f"Final model saved to {final_model_path}")

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

--- Starting Fine-Tuning ---


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
50,3.092100,3.014761
100,3.036100,2.797954
150,2.736700,2.703292
200,1.966200,2.712098
250,2.103800,2.651105
300,2.260500,2.619437
350,1.992200,2.605032
400,1.698800,2.685581
450,1.530600,2.681254
500,1.546000,2.676687


There were missing keys in the checkpoint model loaded: ['output_projection.weight'].


--- Fine-Tuning Complete ---
Final model saved to ./final_biogpt_model


# Compressing and Downloading the Fine-tuned model As ZIP

In [ ]:
import os
import zipfile
from google.colab import files
import shutil

def download_model_as_zip():

    model_folder = "./final_biogpt_model"
    zip_filename = "biogpt_model.zip"

    # Create zip file
    print(f"Compressing {model_folder} into {zip_filename}")
    shutil.make_archive("biogpt_model", 'zip', model_folder)

    # Checking file size
    zip_size = os.path.getsize(zip_filename) / (1024 * 1024)
    print(f"Zip file size: {zip_size:.2f} MB")

    # Downloading the zip file
    print("Starting download")
    files.download(zip_filename)
    print("Download complete! Check your Downloads folder.")

# Run this method
download_model_as_zip()

# Runing Inference On The Test Data

In [10]:
import torch

final_model_path = "./final_biogpt_model"

# Loading the fine-tuned model
model = AutoModelForCausalLM.from_pretrained(final_model_path).to('cuda')

# Loading the tokenizer of our fine-tuned model.
tokenizer = AutoTokenizer.from_pretrained(final_model_path)

In [ ]:
# Loading the test data
test_df = pd.read_csv('test.csv')
test_df.head()

### Starting the inference

In [ ]:
generated_responses = []

print("Starting Inference on Test Data")

# Loop through each row in the test DataFrame
for index, row in test_df.iterrows():
    prompt_text = row['Prompt']

    # formating the input just like the training,
    # The eos_token signals to the model where the prompt ends and generation should begin.
    formatted_prompt = prompt_text + tokenizer.eos_token

    # Tokenizing the formatted prompt and move it to the GPU
    inputs = tokenizer(formatted_prompt, return_tensors='pt').to('cuda')

    # Generate the output from the model
    output_sequences = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=256,  # Max length for the generated response
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # Decoding the generated token IDs back into text
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    # --- Post-processing: Cleaning the output ---
    # The generated text will contain our original prompt. We need to remove it.
    # We find the length of the original prompt and take the substring that comes after it.
    final_response = generated_text[len(prompt_text):].strip()

    # Add the cleaned response to our list
    generated_responses.append(final_response)

    # Print progress every 10 prompts
    if (index + 1) % 10 == 0:
        print(f"Generated response for {index + 1}/{len(test_df)} prompts.")

print("Inference Complete")

In [ ]:
# Adding the generated text to the test dataframe to see how well the text
# accurately answers the prompts
test_df['Generated Response'] = generated_responses

test_df.sample(5)

### From the outputs, it appears the fine-tuned model is prone to hallucination. The model's underlying medical knowledge learned from our small dataset is not robust enough, causing it to generate plausible-sounding but incorrect information.

### Next step after a few hyper-parameter tuning is to try fine-tuning another base model that would work better for our datase
